In [4]:
from desdeo.problem.testproblems.single_objective import new_branin_function, mystery_function, mishras_bird_constrained
from desdeo.emo.hooks.archivers import Archive
from desdeo.emo import algorithms, selection, termination, generator, crossover, mutation, scalar_selection
import polars as pl
import numpy as np
import plotly.graph_objects as go


def run_nsga2_with_mode(
    mode: str,
):
    """Run the NSGA-II style EA once for a given constraint-handling mode."""
    # ---- Problem ----
    problem = new_branin_function()

    nsga2_options = algorithms.nsga2_options()

    constraint_threshold = 10.0
    pop_size = 6
    n_generations = 100
    constraint_symbol = "c_1"
    nsga2_options.template.crossover = crossover.SimulatedBinaryCrossoverOptions(
        xover_probability=0.9, xover_distribution=20
    )
    nsga2_options.template.mutation = mutation.BoundedPolynomialMutationOptions(
        mutation_probability=1.0 / len(problem.variables), distribution_index=20
    )
    nsga2_options.template.mate_selection = scalar_selection.TournamentSelectionOptions(
        name="TournamentSelection", tournament_size=2, winner_size=pop_size
    )
    nsga2_options.template.selection = selection.SingleObjectiveConstrainedRankingSelectorOptions(
        target_objective_symbol="f_1",
        target_constraint_symbol=constraint_symbol,
        constraint_threshold=constraint_threshold,
        population_size=pop_size,
        mode=mode,
    )
    nsga2_options.template.generator = generator.LHSGeneratorOptions(n_points=pop_size)
    nsga2_options.template.termination = termination.MaxGenerationsTerminatorOptions(max_generations=n_generations)

    solver, extras = algorithms.emo_constructor(emo_options=nsga2_options, problem=problem)

    archive = Archive(problem=problem, publisher=extras.publisher)

    extras.publisher.auto_subscribe(archive)
    extras.publisher.register_topics(archive.provided_topics[archive.verbosity], archive.__class__.__name__)

    # ---- Run optimization ----
    results = solver()  # result object not strictly needed; archive holds all solutions

    # Full history of solutions as Polars DataFrame, last population
    return archive.solutions, results


def main():
    modes = ["relaxed", "baseline", "ranking"]

    fig_scatter = go.Figure()

    symbols = {"relaxed": "circle", "baseline": "square", "ranking": "diamond"}

    target_f1 = -268.78792

    for mode in modes:
        to_plot, last = run_nsga2_with_mode(mode=mode)
        # to_plot = last.optimal_outputs
        print(to_plot)

        fig_scatter.add_trace(
            go.Scatter(
                x=to_plot["f_1"],
                y=to_plot["c_1"],
                mode="markers",
                name=f"{mode} (all)",
                opacity=0.6,
                marker={"symbol": symbols[mode]},
            )
        )

        f1_vals = np.asarray(to_plot["f_1"])
        c1_vals = np.asarray(to_plot["c_1"])
        feasible_mask = c1_vals <= 0

        f1_feas = f1_vals[feasible_mask]
        c1_feas = c1_vals[feasible_mask]

        idx = np.argmin(np.abs(f1_feas - target_f1))
        f1_star = float(f1_feas[idx])
        c1_star = float(c1_feas[idx])

        fig_scatter.add_trace(
            go.Scatter(
                x=[f1_star],
                y=[c1_star],
                mode="markers",
                name=f"{mode} highlight (f_1={f1_star:.3f})",
                marker={
                    "symbol": symbols[mode],
                    "size": 7,
                    "line": {"width": 2},
                    "color": "black",
                },
                opacity=1.0,
            )
        )

    fig_scatter.update_layout(
        title="f_1 vs c_1 for different modes",
        xaxis_title="f_1",
        yaxis_title="c_1",
    )

    fig_scatter.show()


main()

shape: (606, 6)
┌───────────┬───────────┬─────────────┬─────────────┬────────────┬────────────┐
│ x_1       ┆ x_2       ┆ f_1         ┆ f_1_min     ┆ c_1        ┆ generation │
│ ---       ┆ ---       ┆ ---         ┆ ---         ┆ ---        ┆ ---        │
│ f64       ┆ f64       ┆ f64         ┆ f64         ┆ f64        ┆ i32        │
╞═══════════╪═══════════╪═════════════╪═════════════╪════════════╪════════════╡
│ 8.06511   ┆ 3.902804  ┆ -126.891561 ┆ -126.891561 ┆ 8.444131   ┆ 1          │
│ -4.646495 ┆ 0.75658   ┆ -417.394825 ┆ -417.394825 ┆ 242.379461 ┆ 1          │
│ 7.264557  ┆ 12.560944 ┆ -13.431644  ┆ -13.431644  ┆ 138.147231 ┆ 1          │
│ 0.597151  ┆ 10.534839 ┆ -108.351235 ┆ -108.351235 ┆ 42.524934  ┆ 1          │
│ 4.679716  ┆ 8.874035  ┆ -65.832868  ┆ -65.832868  ┆ 60.830281  ┆ 1          │
│ …         ┆ …         ┆ …           ┆ …           ┆ …          ┆ …          │
│ -3.165264 ┆ 7.610214  ┆ -227.933106 ┆ -227.933106 ┆ 17.695459  ┆ 100        │
│ -3.215297 ┆ 7.515779  

In [11]:
to_plot

NameError: name 'to_plot' is not defined